In [4]:
cwd = "/Users/jonathanlin/Documents/GitHub/research_transfer/"
csv_path = cwd + "datasets/Animal_Kingdom/action_recognition/annotation/val.csv"

import pandas as pd
import PIL
import random
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from typing import Any, Type

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import wandb

import lightning.pytorch as pl
from lightning.pytorch import Trainer
from torchmetrics import Accuracy
import torch.utils.data as data
from torchvision.datasets import MNIST
from torch.nn import functional as F
import torchvision.models as models
import os
import sys
sys.path.append("/Users/jonathanlin/Documents/GitHub/research_transfer")
from data_tools import ak_ar_images
from data_tools.ak_ar_images import dataloader

In [5]:
df = pd.read_excel("/Users/jonathanlin/Documents/GitHub/research_transfer/datasets/Animal_Kingdom/action_recognition/annotation/df_action.xlsx")
landmarks_frame = pd.read_csv(csv_path, delimiter = " ")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
# caption_model = Blip2ForConditionalGeneration.from_pretrained(
#     "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float32
# )

In [7]:
class clip2_baseline(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()

        # frozen model
        backbone = models.resnet50(weights="DEFAULT")
        backbone.fc = nn.Sequential(
            nn.Dropout(p = 0.2),
            nn.Linear(backbone.fc.in_features, out_features=num_classes)
        )
        self.backbone = backbone

        self.sigm = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigm(self.backbone(x))
    
    def training_step(self, batch, batch_idx):
        images, labels = batch
        # images = images.reshape(-1, 32 * 32)

        # forward pass
        outputs = self(images)
        # loss = nn.BCEWithLogitsLoss()
        loss = nn.BCEWithLogitsLoss()(outputs, labels.type(torch.float32))

        print(loss)
        return {'loss':loss}
    
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        # images = images.reshape(-1, 32 * 32)

        # forward pass
        outputs = self(images)
        loss = nn.BCEWithLogitsLoss()(outputs, labels.type(torch.float32))

        return {'loss':loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = 0.001)
    
    def train_dataloader(self):
        train_loader, val_loader = dataloader.get_data()
        return train_loader
    
    def val_dataloader(self):
        train_loader, val_loader = dataloader.get_data()
        return val_loader
    
epochs = 10
model = clip2_baseline(num_classes = 139)
trainer = Trainer(max_epochs = epochs, fast_dev_run=False)
trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type    | Params
-------------------------------------
0 | backbone | ResNet  | 23.8 M
1 | sigm     | Sigmoid | 0     
-------------------------------------
23.8 M    Trainable params
0         Non-trainable params
23.8 M    Total params
95.171    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

tensor(0.9673, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.9289, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.8616, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.7903, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.7526, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.7239, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.7073, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.7007, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.7028, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6943, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6952, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6957, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6936, device='mps:0', grad_fn=<

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
